In [1]:
import pandas as pd

file_dir = "archive" # Define paths
kaggle_metadata_path = f'{file_dir}/raw_kaggle_df.csv'
movielens_rating_data_path = f'{file_dir}/raw_movielens_rating_df.csv'
wiki_movies_path = f'{file_dir}/clean_wiki_movies_df.csv'

In [2]:
def Cleanr(kaggle_metadata_path, movielens_rating_data_path, wiki_movies_path):
    
    kaggle_metadata = pd.read_csv(kaggle_metadata_path, low_memory=False) # Read csv data
    movielens_rating_data = pd.read_csv(movielens_rating_data_path, low_memory=False)
    wiki_movies_data = pd.read_csv(wiki_movies_path, low_memory=False)

    kaggle_metadata = pd.DataFrame(kaggle_metadata) # Create dataframe from local file path
    movielens_rating_df = pd.DataFrame(movielens_rating_data)
    wiki_movies_df = pd.DataFrame(wiki_movies_data)
    
    # Clean the Kaggle & Movie lengths metadata
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])
    movielens_rating_df['timestamp'] = pd.to_datetime(movielens_rating_df['timestamp'], unit='s')
    movielens_rating_df = movielens_rating_df.rename(columns={"movieId": "id"})
    
    # Merged the wiki movies and kaggle movies DataFrames 
    movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])

    # Drop unnecessary columns from merge
    movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)
    
    # Fill in the missing Kaggle data
    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(
            lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column], axis=1)
        df.drop(columns=wiki_column, inplace=True)
        
    # Call the function with DataFrame and columns as the arguments
    fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
    fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
    fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')
    
    # Filter movies DataFrame for specific columns.
    for col in movies_df.columns:
        lists_to_tuples = lambda x: tuple(x) if type(x) == list else x
        value_counts = movies_df[col].apply(lists_to_tuples).value_counts(dropna=False)
        num_values = len(value_counts)
        if num_values == 1:
            movies_df.drop(columns=['video'], inplace=True)
    
    # Merge the ratings DataFrame.
    movies_df = pd.merge(movies_df, movielens_rating_df, how='left', on = ['id'])  
    
    # Rename and Rearrange new columns         
    movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection',
                                  'url','imdb_link','runtime','budget_kaggle','revenue','release_date_kaggle',
                                  'popularity','vote_average','vote_count','genres','original_language','overview',
                                  'spoken_languages','Country','production_companies','production_countries',
                                  'Distributor','Producer(s)','Director','Starring','Cinematography','Editor(s)',
                                  'Writer(s)','Composer(s)','Based on']]

    movies_df.rename({'id':'kaggle_id','title_kaggle':'title','url':'wikipedia_url','budget_kaggle':'budget',
                      'release_date_kaggle':'release_date','Country':'country','Distributor':'distributor',
                      'Producer(s)':'producers','Director':'director','Starring':'starring',
                      'Cinematography':'cinematography','Editor(s)':'editors','Writer(s)':'writers',
                      'Composer(s)':'composers','Based on':'based_on'}, axis='columns', inplace=True)

    return movies_df


In [3]:
movies_df = Cleanr(kaggle_metadata_path, movielens_rating_data_path, wiki_movies_path)
movies_df.to_csv(f'{file_dir}/cleaned_merged_movies_df.csv', index_label= False)
movies_df.head()

,imdb_id,kaggle_id,title,original_title,tagline,belongs_to_collection,wikipedia_url,imdb_link,runtime,budget,...,production_countries,distributor,producers,director,starring,cinematography,editors,writers,composers,based_on
0,tt0098987,9548,The Adventures of Ford Fairlane,The Adventures of Ford Fairlane,Kojak. Columbo. Dirty Harry. Wimps.,NaN,https://en.wikipedia.org/wiki/The_Adventures_o...,https://www.imdb.com/title/tt0098987/,104.0,49000000.0,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",20th Century Fox,"['Steve Perry', 'Joel Silver']",Renny Harlin,"['Andrew Dice Clay', 'Wayne Newton', 'Priscill...",Oliver Wood,Michael Tronick,"['David Arnott', 'James Cappe']","['Cliff Eidelman', 'Yello']","['Characters', 'by Rex Weiner']"
1,tt0098994,25501,"After Dark, My Sweet","After Dark, My Sweet",All they risked was everything.,NaN,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",https://www.imdb.com/title/tt0098994/,114.0,6000000.0,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",Avenue Pictures,"['Ric Kidney', 'Robert Redlin']",James Foley,"['Jason Patric', 'Rachel Ward', 'Bruce Dern', ...",Mark Plummer,Howard E. Smith,"['James Foley', 'Robert Redlin']",Maurice Jarre,"['the novel', 'After Dark, My Sweet', 'by', 'J..."
2,tt0099005,11856,Air America,Air America,The few. The proud. The totally insane.,NaN,https://en.wikipedia.org/wiki/Air_America_(film),https://www.imdb.com/title/tt0099005/,112.0,35000000.0,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",TriStar Pictures,Daniel Melnick,Roger Spottiswoode,"['Mel Gibson', 'Robert Downey Jr.', 'Nancy Tra...",Roger Deakins,"['John Bloom', 'Lois Freeman-Fox']","['John Eskow', 'Richard Rush']",Charles Gross,"['Air America', 'by', 'Christopher Robbins']"
3,tt0099012,8217,Alice,Alice,NaN,NaN,https://en.wikipedia.org/wiki/Alice_(1990_film),https://www.imdb.com/title/tt0099012/,102.0,12000000.0,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",Orion Pictures,Robert Greenhut,Woody Allen,"['Alec Baldwin', 'Blythe Danner', 'Judy Davis'...",Carlo Di Palma,Susan E. Morse,Woody Allen,NaN,NaN
4,tt0099018,25943,Almost an Angel,Almost an Angel,Who does he think he is?,NaN,https://en.wikipedia.org/wiki/Almost_an_Angel,https://www.imdb.com/title/tt0099018/,95.0,25000000.0,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",Paramount Pictures,John Cornell,John Cornell,"['Paul Hogan', 'Elias Koteas', 'Linda Kozlowski']",Russell Boyd,David Stiven,Paul Hogan,Maurice Jarre,NaN
